In [ ]:
!pip install spotipy

In [ ]:
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials #To accessauthorised Spotify data
import requests
from bs4 import BeautifulSoup
import json
import pandas as pd
import pickle
from collections import deque
from queue import Queue
import time
import concurrent.futures
import os.path

from google.colab import files
uploaded = files.upload()

In [ ]:
with open('artists_uri.pkl', 'rb') as f:
    names = pickle.load(f)

In [ ]:
# Set up Spotify api access and put actual keys
client_id = 'client id'
client_secret = 'client sceret key'

client_credentials_manager = SpotifyClientCredentials(client_id = client_id, client_secret = client_secret)
sp =spotipy.Spotify(client_credentials_manager=client_credentials_manager) #spotify object to access API

In [ ]:
uri_dict = {}

for artist in names:
    result_ = sp.search(art, limit=50) #search query
    artists_list = result_['tracks']['items'][0]['artists']
    for i in range(len(artists_list)): 
        uri = (artists_list[i]['uri'])
        name = (artists_list[i]['name'])
        uri_dict[name] = uri


In [ ]:
artist_uri_df = pd.DataFrame(uri_dict.items(), columns = ['artist_name', 'uri'])

In [ ]:
artist_uri_df

In [ ]:
uri_list = list(artist_uri_df['uri'])
uri_deque = deque(list(artist_uri_df['uri']))


In [ ]:
len(uri_list)

In [ ]:
def get_related_artists(uri):
    related = sp.artist_related_artists(uri)
    uris=[]
    for i in range(len(related['artists'])):
        uris.append(related['artists'][i]['uri'])
        #related_artists[name] = uri
    return (uris)

Get 
List of 10 000 artist uris 

In [ ]:
#get list of 10 000 artist uris by getting related artists recursively
t1 = time.perf_counter()
while len(uri_list) <= 10_000:
    uri = uri_deque.popleft()
    returned_uris =  get_related_artists(uri)
    (uri_list.extend(returned_uris)) 
    uri_list = list(set(uri_list))
    uri_deque.extend(returned_uris)
    print(len(uri_list))
    
t2 = time.perf_counter()
print(f'Process took { t2- t1} seconds to complete.')

In [ ]:
#Save artist uri's in a pickle file

with open('full_artists_uri.pkl', 'wb') as f:
    pickle.dump(uri_list, f)

GET ALBUM URIS

In [ ]:
def get_artist_albums(uri):
    names=[]
    uris=[]
    results = sp.artist_albums(uri, album_type='album', country='us')
    albums = results['items']
    while results['next']:
        results = sp.next(results)
        albums.extend(results['items'])
        
    for album in albums:
        names.append(album['name'])
        uris.append(album['uri'])
        #album_artist.append(album[''])
        #print(album)
    names = list(set(names))
    uris = list(set(uris))
    return uris

In [ ]:
album_uris = []
i=0
for uri in uri_list:
    i +=1 
    album_uris.extend(get_artist_albums(uri))
    if (i % 100 == 0):
        time.sleep(5)
        print(i)
    else:
        continue
    
 

In [ ]:
album_uris = list(set(album_uris))
print(len(album_uris))

In [ ]:
#save all album uris as pickle
with open('album_uris.pkl', 'wb') as f:
    pickle.dump(album_uris, f)

**GET ALBUM TRACK URIS and SONG DATA**


In [ ]:
def get_album_tracks(album_uri):
    tracks= sp.album_tracks(album_uri, market='us')
    return tracks


In [ ]:
def get_song_data(track):
    song_data_dict ={}
    song_data_dict['artist_name'] = album_tracks['items'][0]['artists'][0]['name']
    song_data_dict['album_name'] = album_tracks['items'][0]['name']
    song_data_dict['track_title'] = track['name']
    song_data_dict['track_uri'] = track['uri']
    song_data_dict['track_preview_url'] = track['preview_url']

    return song_data_dict

In [ ]:
#download data and save after 1000 albums 
song_data_list = []
counter=0
for album_uri in album_uris[10:]:

    album_tracks= sp.album_tracks(album_uri, market='us')
    for track in (album_tracks['items']):
        song_data_list.append(get_song_data(track))

    counter += 1
    
    if counter % 1000 == 0:
        with open(f'song_data_{counter}.pkl', 'wb') as f:
            pickle.dump(song_data_list, f)
        print(f'saved{counter}')


In [ ]:
pd.DataFrame(song_data_list)

In [ ]:
song_data_df = pd.DataFrame(song_data_list)
song_data_df.to_csv('song_data.csv',index=False)

DOWNLOAD  MP3 PREVIEW FILES

In [ ]:
from google.colab import files
uploaded = files.upload()

In [ ]:
with open('mp3_urls_dict.pkl', 'rb') as f:
    mp3_urls_dict = pickle.load(f)

In [ ]:
my_headers = {
'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 ' + ' (KHTML, like Gecko) Chrome/61.0.3163.100Safari/537.36'
}

In [ ]:

save_path = 'Music/'
def download_file(url,title):
    local_filename = url.split('/')[-1]
    name_of_file = title

    completeName = os.path.join(save_path, name_of_file +'.mp3')   
    # NOTE the stream=True parameter below
    with requests.get(url, headers = my_headers, stream=True) as r:
        r.raise_for_status()
        with open(completeName +'.mp3', 'wb') as f:
            for chunk in r.iter_content(chunk_size=8192): 
                # If you have chunk encoded response uncomment if
                # and set chunk_size parameter to None.
                #if chunk: 
                f.write(chunk)
    return local_filename

In [ ]:
my_headers = {
'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 ' + ' (KHTML, like Gecko) Chrome/61.0.3163.100Safari/537.36'
}

for i,j in mp3_urls_dict.items():
    download_file(j,i)
    
#Convert Music Folder into Zip
!zip -r /content/Music.zip /content/Music

In [ ]:
t1= time.perf_counter()
with concurrent.futures.ThreadPoolExecutor() as executor:
    results = executor.map(download_file,song_data_list)
t2= time.perf_counter()
print(f'Process took { t2- t1} seconds to complete.')

In [ ]:
#a = []
b = []
for i in range(100,(len(track_uri_list)+100), 100):
    b.append(i)

In [ ]:
audio_features =[]
for i,j in zip(a,b):
    audio_features.extend(sp.audio_features(tracks=track_uri_list[i:j]))

In [ ]:
audio_features

In [ ]:
audio_features_ = pd.DataFrame(audio_features)

In [ ]:
audio_features_df